#### **Liverpool, Mancity, and Chelsea last 7 years matches**


Using Fbref website, my goal is to extract all matches that liverpool, Mancity, Chelsea played during 2018:2025


In [ ]:
from bs4 import BeautifulSoup as bs 
from urllib.request import urlopen

In [ ]:
url="https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats"

In [ ]:
client =urlopen(url)

In [ ]:
html= client.read()

In [ ]:
html

In [ ]:
client.close()
soup = bs(html, "html.parser")  
soup

In [ ]:
import pandas as pd
df=pd.read_html('https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats',attrs={"id":"matchlogs_for"})
df[0].head()

In [ ]:
import pandas as pd
base_url = "https://fbref.com/en/squads/cff3d9bb/{season}/Chelsea-Stats"

seasons = [
    "2018-2019",
    "2019-2020",
    "2020-2021",
    "2021-2022",
    "2022-2023",
    "2023-2024",
    "2024-2025"  
]

all_data = []

for season in seasons:
    url = base_url.format(team_id='cff3d9bb', season=season)
    try:
        print(f"Scraping: {season}")
        tables = pd.read_html(url, attrs={"id": "matchlogs_for"})
        if tables:
            df = tables[0]
            df["Season"] = season  
            all_data.append(df)
    except Exception as e:
        print(f"Failed for {season}: {e}")


full_df = pd.concat(all_data, ignore_index=True)
full_df.to_csv("chelsea_18to25.csv", index=False)



In [ ]:
import pandas as pd
base_url = "https://fbref.com/en/squads/b8fd03ef/{season}/Manchester-City-Stats"

seasons = [
    "2018-2019",
    "2019-2020",
    "2020-2021",
    "2021-2022",
    "2022-2023",
    "2023-2024",
    "2024-2025"  
]

all_data = []

for season in seasons:
    url = base_url.format(team_id='b8fd03ef', season=season)
    try:
        print(f"Scraping: {season}")
        tables = pd.read_html(url, attrs={"id": "matchlogs_for"})
        if tables:
            df = tables[0]
            df["Season"] = season  
            all_data.append(df)
    except Exception as e:
        print(f"Failed for {season}: {e}")


full_df = pd.concat(all_data, ignore_index=True)
full_df.to_csv("Manchester-City_18to25.csv", index=False)



In [ ]:
import pandas as pd
base_url = "https://fbref.com/en/squads/822bd0ba/{season}/Liverpool-Stats"

seasons = [
    "2018-2019",
    "2019-2020",
    "2020-2021",
    "2021-2022",
    "2022-2023",
    "2023-2024",
    "2024-2025"  
]

all_data = []

for season in seasons:
    url = base_url.format(team_id='822bd0ba', season=season)
    try:
        print(f"Scraping: {season}")
        tables = pd.read_html(url, attrs={"id": "matchlogs_for"})
        if tables:
            df = tables[0]
            df["Season"] = season  
            all_data.append(df)
    except Exception as e:
        print(f"Failed for {season}: {e}")


full_df = pd.concat(all_data, ignore_index=True)
full_df.to_csv("Liverpool_18to25.csv", index=False)



In [ ]:
import pandas as pd
df1 = pd.read_csv('Liverpool_18to25.csv')
df2 = pd.read_csv('Manchester-City_18to25.csv')
df3 = pd.read_csv('chelsea_18to25.csv')

venue_index = df1.columns.get_loc('Venue')  
df1.insert(venue_index + 1, 'Club', 'Liverpool')  
df2.insert(venue_index + 1, 'Club', 'Manchester City') 
df3.insert(venue_index + 1, 'Club', 'Chelsea')  

combined_df = pd.concat([df1, df2, df3], ignore_index=True)
combined_df.to_csv('LIV-CHE-MAN.csv', index=False)

sort the based on date


In [61]:
df=pd.read_csv('LIV-CHE-MAN.csv')
df=df.sort_values(by=['Date'], ascending=True)
df.to_csv(('LIV-CHE-MAN.csv'), index=False)

In [ ]:
from datetime import datetime
today = pd.Timestamp(datetime.today().date())

df['Date'] = pd.to_datetime(df['Date'])
df = df[df['Date'] < today]

df.to_csv('LIV-CHE-MAN.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('LIV-CHE-MAN.csv',encoding='utf-8')

df['GF'] = df['GF'].astype(str)
df['GA'] = df['GA'].astype(str)

df['GF'] = df['GF'].str.extract(r'(\d+)').astype(int)
df['GA'] = df['GA'].str.extract(r'(\d+)').astype(int)

xg_means_by_GF = df.groupby('GF')['xG'].mean()
df['xG'] = df.apply(lambda row: xg_means_by_GF.get(row['GF'], df['xG'].mean()) if pd.isna(row['xG']) else row['xG'], axis=1)

xga_means_by_GA = df.groupby('GA')['xGA'].mean()
df['xGA'] = df.apply(lambda row: xga_means_by_GA.get(row['GA'], df['xGA'].mean()) if pd.isna(row['xGA']) else row['xGA'], axis=1)
clubs = ['Liverpool', 'Chelsea', 'Manchester City']

i = 0
while i < len(df) - 1:
    row = df.iloc[i]
    next_row = df.iloc[i + 1]
    
    if row['Opponent'] in clubs and next_row['Opponent'] in clubs:
        df.at[i + 1, 'xG'] = row['xGA']
        df.at[i + 1, 'xGA'] = row['xG']
        i += 2 
        # Skip the next row since we just updated it
    else:
        i += 1

df.to_csv('LIV-CHE-MAN.csv', index=False,encoding='utf-8')


In [ ]:
df['xG'] = df['xG'].round(2)
df['xGA'] = df['xGA'].round(2)
df.to_csv('LIV-CHE-MAN.csv', index=False, encoding='utf-8')


In [ ]:
season_col = df.pop('Season')  
df.insert(0, 'Season', season_col) 
df.to_csv('LIV-CHE-MAN.csv', index=False, encoding='utf-8')

In [7]:
import pandas as pd
df=pd.read_csv('LIV-CHE-MAN.csv',encoding='utf-8')
df['Formation'] = df['Formation'].astype(str)

df.to_excel('LIV-CHE-MAN.xlsx', index=False)


In [8]:
import pandas as pd
df = pd.read_excel('LIV-CHE-MAN.xlsx')
df.loc[df['Round'] == 'Final', 'Venue'] = 'Neutral'
df.to_excel('LIV-CHE-MAN.xlsx', index=False)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1181 entries, 0 to 1180
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Season         1181 non-null   object 
 1   Date           1181 non-null   object 
 2   Time           1181 non-null   object 
 3   Comp           1181 non-null   object 
 4   Round          1181 non-null   object 
 5   Day            1181 non-null   object 
 6   Venue          1181 non-null   object 
 7   Club           1181 non-null   object 
 8   Result         1181 non-null   object 
 9   GF             1181 non-null   int64  
 10  GA             1181 non-null   int64  
 11  Opponent       1181 non-null   object 
 12  xG             1181 non-null   float64
 13  xGA            1181 non-null   float64
 14  Poss           1179 non-null   float64
 15  Attendance     981 non-null    float64
 16  Captain        1181 non-null   object 
 17  Formation      1181 non-null   object 
 18  Opp Form

In [ ]:
import pandas as pd

df= pd.read_excel('LIV-CHE-MAN.xlsx')
df['Attendance'] = df['Attendance'].fillna(0)
print(df['Attendance'].isna().sum())  


0


In [2]:
import plotly.express as px
import pandas as pd
df= pd.read_excel('LIV-CHE-MAN.xlsx')
fig = px.scatter(
    df,
    x='xG',  
    y='GF', 
    color='Comp',  # Color by competition
    hover_name='Club',  hover_data=['Opponent'],
    title='Goals Scored vs Expected Goals (xG)',
    labels={'xG': 'Expected Goals (xG)', 'GF': 'Goals Scored (GF)'},
    template='plotly_dark',
    color_discrete_sequence=px.colors.qualitative.Set1  # Use a color palette
)

fig.update_layout(
    height=600,
    width=950,
    title_font_size=20,
    font=dict(family='Arial', size=14),
    xaxis_title='Expected Goals (xG)',
    yaxis_title='Goals Scored (GF)',
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)',  
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


In [ ]:
import plotly.express as px

result_counts = df.groupby(['Club', 'Result']).size().reset_index(name='Count')
result_counts['Result'] = result_counts['Result'].str.strip()  

fig = px.bar(
    result_counts,
    x='Club', 
    y='Count',  
    color='Result',  
    title='Match Results by Team',
    labels={'Count': 'Number of Matches', 'Club': 'Team', 'Result': 'Result'},
    template='plotly_dark',
    color_discrete_map={'Win': 'green', 'Loss': 'red', 'Draw': 'gray'} 
)

fig.update_layout(
    height=600,
    width=950,
    title_font_size=20,
    font=dict(family='Arial', size=14),
    xaxis_title='Team',
    yaxis_title='Number of Matches',
    xaxis_tickangle=-45,
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)',  
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


In [ ]:
import plotly.express as px

fig = px.histogram(
    df,
    x='Attendance',  
    nbins=20,  
    title='Distribution of Match Attendance',
    labels={'Attendance': 'Attendance'},
    template='plotly_dark',
)


fig.update_layout(
    height=600,
    width=950,
    title_font_size=20,
    font=dict(family='Arial', size=14),
    xaxis_title='Attendance',
    yaxis_title='Frequency',
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)',  
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

df= pd.read_excel('LIV-CHE-MAN.xlsx')
df.loc[df['Venue'] == 'Home', 'Attendance'] = df['Attendance'].fillna(0)

df_home_games = df[df['Venue'] == 'Home']

df_home_games['Date'] = pd.to_datetime(df_home_games['Date'])
top_teams = df_home_games['Club'].value_counts().head(3).index
df_top_teams_home = df_home_games[df_home_games['Club'].isin(top_teams)]

fig = px.line(
    df_top_teams_home,
    x='Date',  
    y='Attendance', 
    color='Club',  
    title='Change in Home Game Attendance Over Time for Top 3 Teams',
    labels={'Date': 'Date', 'Attendance': 'Attendance'},
    template='plotly_dark',
)

fig.update_layout(
    height=600,
    width=950,
    title_font_size=20,
    font=dict(family='Arial', size=14),
    xaxis_title='Date',
    yaxis_title='Attendance',
    xaxis_tickangle=-45,
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)', 
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


C:\Users\HP\AppData\Local\Temp\ipykernel_21476\396488802.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import plotly.express as px
fig = px.scatter(
    df,
    x='GF', 
    y='GA',  
    color='Comp',  
    hover_name='Club', 
    title='Goals Scored vs Goals Against by Team',
    labels={'GF': 'Goals Scored', 'GA': 'Goals Against'},
    template='plotly_dark',
    color_discrete_sequence=px.colors.qualitative.Set1  
)

fig.update_layout(
    height=600,
    width=950,
    title_font_size=20,
    font=dict(family='Arial', size=14),
    xaxis_title='Goals Scored (GF)',
    yaxis_title='Goals Against (GA)',
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)',  
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


In [ ]:
import plotly.express as px
xg_avg = df.groupby('Comp')['xG'].mean().reset_index()

fig = px.bar(
    xg_avg,
    x='Comp',  
    y='xG',  
    title='Average xG by Competition',
    labels={'xG': 'Average xG', 'Comp': 'Competition'},
    template='plotly_dark',
)
fig.update_layout(
    height=600,
    width=950,
    title_font_size=20,
    font=dict(family='Arial', size=14),
    xaxis_title='Competition',
    yaxis_title='Average xG',
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)', 
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

liverpool_df = df[df['Club'] == 'Liverpool'].copy()
referee_result_counts = liverpool_df.groupby(['Referee', 'Result']).size().reset_index(name='Count')

top_referees = referee_result_counts.groupby('Referee')['Count'].sum().sort_values(ascending=False).head(5).index
referee_result_counts = referee_result_counts[referee_result_counts['Referee'].isin(top_referees)]

total_matches_by_referee = referee_result_counts.groupby('Referee')['Count'].transform('sum')
referee_result_counts['Percentage'] = (referee_result_counts['Count'] / total_matches_by_referee) * 100

fig = px.bar(
    referee_result_counts,
    x='Referee',
    y='Percentage',
    color='Result',
    barmode='group',
    title='Liverpool Match Results (%) with Top Referees',
    labels={'Percentage': 'Percentage (%)', 'Referee': 'Referee', 'Result': 'Result'},
    template='plotly_dark',
    color_discrete_map={'W': 'green', 'D': 'gray', 'L': 'red'}
)

fig.update_layout(
    height=600,
    width=1100,
    title_font_size=22,
    font=dict(family='Arial', size=14),
    xaxis_title='Referee',
    yaxis_title='Percentage (%)',
    xaxis_tickangle=-45,
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)',
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

liverpool_df = df[df['Club'] == 'Manchester City'].copy()
referee_result_counts = liverpool_df.groupby(['Referee', 'Result']).size().reset_index(name='Count')

top_referees = referee_result_counts.groupby('Referee')['Count'].sum().sort_values(ascending=False).head(5).index
referee_result_counts = referee_result_counts[referee_result_counts['Referee'].isin(top_referees)]

total_matches_by_referee = referee_result_counts.groupby('Referee')['Count'].transform('sum')
referee_result_counts['Percentage'] = (referee_result_counts['Count'] / total_matches_by_referee) * 100

fig = px.bar(
    referee_result_counts,
    x='Referee',
    y='Percentage',
    color='Result',
    barmode='group',
    title='Man City Match Results (%) with Top Referees',
    labels={'Percentage': 'Percentage (%)', 'Referee': 'Referee', 'Result': 'Result'},
    template='plotly_dark',
    color_discrete_map={'W': 'green', 'D': 'gray', 'L': 'red'}
)

fig.update_layout(
    height=600,
    width=1100,
    title_font_size=22,
    font=dict(family='Arial', size=14),
    xaxis_title='Referee',
    yaxis_title='Percentage (%)',
    xaxis_tickangle=-45,
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)',
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

liverpool_df = df[df['Club'] == 'Chelsea'].copy()
referee_result_counts = liverpool_df.groupby(['Referee', 'Result']).size().reset_index(name='Count')

top_referees = referee_result_counts.groupby('Referee')['Count'].sum().sort_values(ascending=False).head(5).index
referee_result_counts = referee_result_counts[referee_result_counts['Referee'].isin(top_referees)]

total_matches_by_referee = referee_result_counts.groupby('Referee')['Count'].transform('sum')
referee_result_counts['Percentage'] = (referee_result_counts['Count'] / total_matches_by_referee) * 100
fig = px.bar(
    referee_result_counts,
    x='Referee',
    y='Percentage',
    color='Result',
    barmode='group',
    title='Chelsea Match Results (%) with Top Referees',
    labels={'Percentage': 'Percentage (%)', 'Referee': 'Referee', 'Result': 'Result'},
    template='plotly_dark',
    color_discrete_map={'W': 'green', 'D': 'gray', 'L': 'red'}
)
fig.update_layout(
    height=600,
    width=1100,
    title_font_size=22,
    font=dict(family='Arial', size=14),
    xaxis_title='Referee',
    yaxis_title='Percentage (%)',
    xaxis_tickangle=-45,
    margin=dict(l=60, r=40, t=80, b=60),
    plot_bgcolor='rgb(26, 26, 26)',
    paper_bgcolor='rgb(26, 26, 26)',
)

fig.show()


In [9]:
import pandas as pd
df= pd.read_excel('LIV-CHE-MAN.xlsx')
df['Attendance'] = df['Attendance'].fillna(0)
df['Attendance'] = df['Attendance'].astype(int)
df.to_excel('LIV-CHE-MAN.xlsx', index=False)